In [218]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64
import random

In [290]:
def key_generator_AES(kp):
    """
    generer des clefs de 128 bits avec kp bits significatifs
    et le reste bourré avec des 0
    kp : doit etre multiple de 8 sinon on prend la multiple de 8
    la plus proche inf
    """
    key = get_random_bytes(kp//8).ljust(16,b'0')
    
    return (key)

In [291]:
def double_AES(msg, key1, key2,  mode = 'MODE_ECB'):
    """
    faire le double chiffrement AES en ECB 
    en passant les 2 clefs en parametres
    """
    if not (isinstance(msg,bytes)):
        msg = msg.encode()
            
    enc = simple_enc_AES(msg, key1)

    enc = simple_enc_AES(enc, key2)

    return enc

    
def simple_enc_AES(msg, key, mode = 'MODE_ECB'):
    """
    chiffrer AES
    """
    cipher = AES.new(key, AES.MODE_ECB)
    enc = cipher.encrypt(pad(msg, len(key)))
    return enc

def simple_dec_AES(msg, key, mode = 'MODE_ECB'):
    """
    dechiffrer AES
    """
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(msg)
    return unpad(plaintext, len(key))

In [316]:
kp = 32
key1 = key_generator_AES(kp)
key2 = key_generator_AES(kp)
M1 = "Voici le message 1"
C1 = double_AES(M1,key1,key2)
M2 = "Voici le message 2"
C2 = double_AES(M2,key1,key2)

In [313]:
'''x0 = get_random_bytes(4)
l = list(x0)
bin(l[-2])'''
print()

In [312]:
'''def shuff_fuc(x0, kp):
    """
    retourne un couple ()
    """
    res  = list(x0)
    random.shuffle(res)
    res2 = bytes(res[:kp]).ljust(16,b'0')
    return bytes(res),  res2

'''
print()

In [314]:
def trail( f, msg ,  n , kp, l):
    """
    retourne un triplet (x0, xd, d) 
    """
    
    x0 = key_generator_AES(kp)
    tmp = x0
    d = 0
    max_it = 20/ (1/(2**l))  # d'apres l'article
    # 2**(l+2)

    
    #mask_l = 2**l - 1  cond d'arrret ? 
    while True:
        c_temp = f(msg, tmp)
    
        if d == max_it:
            #print("Risque de cycle")
            return None
        # Est ce que la condition d'arret est sur la clef ou sur le 
        # le resultat de l'application de la fonction ? 
        '''
            if   == 0:  # condition d'arret
            xd = tmp
            break
            '''
        d += 1
        tmp = c_temp[:kp//8].ljust(16,b'0')
        
    
    return (x0, xd, d)